# Mathematical Optimisation - Project
This project aims at ricreating the results reported in:

[A fix-and-optimize heuristic for the Unrelated Parallel Machine Scheduling Problem
](https://www.sciencedirect.com/science/article/pii/S0305054823003684)

In [ ]:
from Solver import Solver
from MPA.MPA import MPA
from Heuristic.Greedy import Greedy
from Heuristic.FixOpt import FixOpt
import random as r
import json
from IPython.display import clear_output

r.seed(3000)
  
with open('credentials.txt') as f: 
    data = f.read() 
options = json.loads(data)

## Creation of the matrices

- P stores the execution times
- S stores the setup times

These matrices are converted to dictionaries to handle the indexes not starting from 0

In [ ]:
def generate_instance(N_cardinality, M_cardinality,
                      min_execution_time = 1,max_execution_time = 10,
                      min_setup_time = 1,max_setup_time = 3):
    
    P = {}
    S = {}
    
    N = range(1, N_cardinality+1)
    M = range(1, M_cardinality+1)
    
    for i in M:
        for j in N:
            P[i,j] = r.randint(min_execution_time, max_execution_time)
            
    for i in M:
        for j in N:
            for k in N:
                S[i,j,k] = r.randint(min_setup_time, max_setup_time)
    
    return P,S

N_cardinality = 30
M_cardinality = 5

P_dict, S_dict = generate_instance(N_cardinality=N_cardinality,M_cardinality=M_cardinality)
print(f'P: {P_dict}')
print(f'S: {S_dict}')

In [ ]:
N = range(1, N_cardinality+1)
M = range(1, M_cardinality+1)
N0 = [i for i in N]
N0.insert(0,0)

# Exact Solution

The problem is solved exactly using the Solver class.

Then, the results are printed. Note that job 0 is a dummy job used to represent the beginning and the end of activities for each machine.

In [ ]:
s = Solver(execution_times = P_dict, setup_times = S_dict)
decision_variables,completion_times,maximum_makespan,assignments = s.solve(options=options)

for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.N:
    print(f'Job {i} completed at {completion_times[i]}s')

print(f'The makespan has been minimized to {maximum_makespan}s')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

# Mathematical Programming Algorithm (MPA)

The paper cites a decomposition algorithm (Branch-and-check) based on the previous formulation.

It is an exact formulation based on a master algorithm and a sequence algorithm:
- The master algorithm is responsible of assigning jobs to machines
- The sequence problem is responsible of finding the optimal sequence of jobs for each machine

In [ ]:
# TODO: fix this part

s = MPA(execution_times = P_dict, setup_times = S_dict)
decision_variables,maximum_makespan = s.solve(options=options)
for i in s.M:
    for j in s.N:
        for k in s.N0:
            if decision_variables[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

print(f'The makespan has been minimized to {maximum_makespan}s')

# Fix-and-Optimize approach

In [ ]:
greedy = Greedy(execution_times=P_dict, setup_times=S_dict)
solution = greedy.solve()

for i in s.M:
    for j in s.N0:
        for k in s.N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled in sequence of job {k} in machine {i}')

for i in s.M:
    for k in s.N:
        if assignments[i,k] == 1:
            print(f'Job {k} has been assigned to machine {i}')

for j in s.N:
    print(f'Job {j} completed at {completion_times[j]}s')

In [ ]:
m = FixOpt(initial_solution=solution, setup_times=S_dict,execution_times=P_dict, N=N, M=M, N0=N0, 
           subproblem_size_adjust_rate=0.1, t_max = 30, subproblem_runtime_limit=10, subproblem_size=5)

solution, makespan = m.solve()
clear_output(wait=True)
for i in M:
    for j in N0:
        for k in N0:
            if solution[i,j,k] == 1:
                print(f'Job {j} scheduled after job {k} in machine {i}')
print(f'The makespan has been minimized to {makespan}s')